In [12]:
# view data

import pandas as pd
import yaml
import os
import datetime
import random
from tensorflow.keras.metrics import BinaryAccuracy, CategoricalAccuracy, Precision, Recall, AUC
from tensorflow.keras.models import save_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from models.models import *
from visualization.visualize import *
from custom.metrics import F1Score
from data.preprocess import remove_text

In [5]:
cfg = yaml.full_load(open("/home/ubuntu/covid-cxr/config.yml", 'r'))
print("1",cfg)
cfg['TRAIN']['EXPERIMENT_TYPE'] #single train

1 {'PATHS': {'RAW_DATA': '/home/ubuntu/covid-cxr/data/', 'MILA_DATA': '/home/ubuntu/covid-cxr/data/covid-chestxray-dataset/', 'FIGURE1_DATA': '/home/ubuntu/covid-cxr/data/Figure1-COVID-chestxray-dataset/', 'RSNA_DATA': '/home/ubuntu/covid-cxr/data/rsna/', 'PROCESSED_DATA': 'data/processed/', 'TRAIN_SET': '/home/ubuntu/covid-cxr/data/processed/train_set.csv', 'VAL_SET': '/home/ubuntu/covid-cxr/data/processed/val_set.csv', 'TEST_SET': '/home/ubuntu/covid-cxr/data/processed/test_set.csv', 'IMAGES': '/home/ubuntu/covid-cxr/documents/generated_images/', 'LOGS': '/home/ubuntu/covid-cxr/results/logs/', 'MODEL_WEIGHTS': '/home/ubuntu/covid-cxr/results/models/', 'MODEL_TO_LOAD': '/home/ubuntu/covid-cxr/results/models/model20201102-033225.h5', 'LIME_EXPLAINER': '/home/ubuntu/covid-cxr/data/interpretability/lime_explainer.pkl', 'OUTPUT_CLASS_INDICES': '/home/ubuntu/covid-cxr/data/interpretability/output_class_indices.pkl', 'BATCH_PRED_IMGS': '/home/ubuntu/covid-cxr/data/processed/test/', 'BATCH_P

'single_train'

In [6]:
data = {}
data['TRAIN'] = pd.read_csv(cfg['PATHS']['TRAIN_SET'])
data['VAL'] = pd.read_csv(cfg['PATHS']['VAL_SET'])
data['TEST'] = pd.read_csv(cfg['PATHS']['TEST_SET'])

In [8]:
# print(data)

In [13]:

# If set in config file, oversample the minority class
if cfg['TRAIN']['IMB_STRATEGY'] == 'random_oversample':
    data['TRAIN'] = random_minority_oversample(data['TRAIN'])

# Create ImageDataGenerators
train_img_gen = ImageDataGenerator(rotation_range=10, preprocessing_function=remove_text,
                                   samplewise_std_normalization=True, samplewise_center=True)
val_img_gen = ImageDataGenerator(preprocessing_function=remove_text,
                                   samplewise_std_normalization=True, samplewise_center=True)
test_img_gen = ImageDataGenerator(preprocessing_function=remove_text,
                                   samplewise_std_normalization=True, samplewise_center=True)

# Create DataFrameIterators
img_shape = tuple(cfg['DATA']['IMG_DIM'])
y_col = 'label_str'
class_mode = 'categorical'
train_generator = train_img_gen.flow_from_dataframe(dataframe=data['TRAIN'], directory=cfg['PATHS']['RAW_DATA'],
    x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode, validate_filenames=False)
val_generator = val_img_gen.flow_from_dataframe(dataframe=data['VAL'], directory=cfg['PATHS']['RAW_DATA'],
    x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode, validate_filenames=False)
test_generator = test_img_gen.flow_from_dataframe(dataframe=data['TEST'], directory=cfg['PATHS']['RAW_DATA'],
    x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
    class_mode=class_mode, validate_filenames=False, shuffle=False)

Found 1489 non-validated image filenames belonging to 2 classes.
Found 146 non-validated image filenames belonging to 2 classes.
Found 182 non-validated image filenames belonging to 2 classes.


In [18]:
type(train_generator)

tensorflow.python.keras.preprocessing.image.DataFrameIterator

In [20]:
dim(train_generator)

NameError: name 'dim' is not defined